In [1]:
from pymongo import MongoClient
from sklearn.model_selection import train_test_split
from keras.wrappers.scikit_learn import KerasRegressor
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import GaussianNoise
from keras.utils import plot_model
from keras.callbacks import TensorBoard
from keras.utils import vis_utils as vizu
from keras.layers import Dropout
from keras import callbacks
from keras import backend
from scipy.integrate import simps
from scipy.stats import f
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import f_regression, mutual_info_regression

C:\Users\Liuya\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def knn_baseline_model():
    
    def root_mean_squared_error(y_true, y_pred):
        return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))
    
    # create model
    model = Sequential()
    #model.add(GaussianNoise(stddev=0.1, input_shape=(14,)))
    model.add(Dense(7, activation='relu', input_shape=(18,)))
    model.add(Dropout(0.5))
    model.add(GaussianNoise(0.1))
    model.add(Dense(24, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss=root_mean_squared_error, optimizer='adam')
    #model.compile(loss='mse', optimizer='adam')
    return model

In [3]:
def coeff_determination(y_true, y_pred):
    ssres = np.sum(np.square(y_true - y_pred))
    sstot = np.sum(np.square(y_true - np.mean(y_pred)))
    return 1 - ssres / sstot

In [4]:
def REC(y_true , y_pred):
    
    Accuracy = []
    Begin_Range = 0
    End_Range = 1.5
    Interval_Size = 0.01
    
    # List of epsilons
    Epsilon = np.arange(Begin_Range , End_Range , Interval_Size)
    
    # Main Loops
    for i in range(len(Epsilon)):
        count = 0.0
        for j in range(len(y_true)):
            y_true[j]=float(y_true[j])
            y_true[j]
            y_pred[j]
            
            np.linalg.norm(y_true[j] - y_pred[j])
            np.sqrt( np.linalg.norm(y_true[j]) **2 + np.linalg.norm(y_pred[j])**2 ) < Epsilon[i]
            
            if np.linalg.norm(y_true[j] - y_pred[j]) / np.sqrt( np.linalg.norm(y_true[j]) **2 + np.linalg.norm(y_pred[j])**2 ) < Epsilon[i]:
                count = count + 1
        
        Accuracy.append(count/len(y_true))
    
    # Calculating Area Under Curve using Simpson's rule
    AUC = simps(Accuracy , Epsilon ) / End_Range
        
    # returning epsilon , accuracy , area under curve    
    return Epsilon, Accuracy, AUC

In [5]:
def model_evaluate (model, X_train, X_test, y_train, y_test):

    y_pred = model.predict(X_test)
    Deviation, Accuracy, auc = REC(y_test.values, y_pred)
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    #r2 = r2_score(y_test, y_pred)
    r2 = coeff_determination(y_test, y_pred)
    mape = ((y_test - y_pred) / y_test).abs().mean()*100
    
    f_test = np.var(y_test) / np.var(y_pred)
    df1 = len(y_test) - 1
    df2 = len(y_pred) - 1
    p_value = 1 - 2 * abs(0.5 - f.cdf(f_test, df1, df2))
    
    p_stars = ''
    if p_value <= 0.05:
        p_stars = '*'
    if p_value <= 0.01:
        p_stars = '**'
    if p_value <= 0.001:
        p_stars = '***'
    #print(F, p_value)
    
    y_validate = model.predict(X_train)
    training_loss = sqrt(mean_squared_error(y_train, y_validate))
    
    return_dict = {'rmse':rmse, 
                   'mape':mape, 
                   'r2':r2, 
                   'auc':auc,
                   'training_loss':training_loss, 
                   'f_test':f_test,
                   'p':p_stars,
                   'y_pred':y_pred, 
                   'y_validate':y_validate}
    
    return return_dict

In [6]:
conn = MongoClient('140.115.53.147', 27017)

In [7]:
db = conn['NCU_CHKA']
collection = db['chka_all_features_by_week']
all_features = pd.DataFrame(list(collection.find()))
all_features = all_features.drop(['_id'], axis=1)
collection = db['chka_final_score']
final_score = pd.DataFrame(list(collection.find()))
final_score = final_score.drop(['_id'], axis=1)
all_features.head()

,Cumulative_num_days,Problem_num_day,active_num_days,active_sum_count,homework_complete_num,homework_num,num_complete,num_watched,pause_video_sum,seek_video_sum,stop_video_sum,username,video_backward_seek_sum,video_forward_seek_sum,video_num_days,video_pause_sum,video_stop_sum,video_sum_count,week
0,0,0,3,318,0,0,0,4,4,3,0,Y1050160,3,6,3,9,0,25,2
1,0,0,2,2,0,0,0,0,0,0,0,Y1050161,0,0,0,0,0,0,3
2,0,0,0,0,0,0,0,0,0,0,0,Y1050161,0,0,0,0,0,0,4
3,0,0,0,0,0,0,0,0,0,0,0,Y1050161,0,0,0,0,0,0,5
4,0,0,1,231,0,0,0,0,0,0,0,Y1050161,0,0,0,0,0,0,2


In [8]:
chka = pd.merge(all_features, final_score,on='username')
chka.fillna(0.0, inplace=True)
chka = chka.drop(['username'], axis=1)
chka.head(3)

,Cumulative_num_days,Problem_num_day,active_num_days,active_sum_count,homework_complete_num,homework_num,num_complete,num_watched,pause_video_sum,seek_video_sum,stop_video_sum,video_backward_seek_sum,video_forward_seek_sum,video_num_days,video_pause_sum,video_stop_sum,video_sum_count,week,final_score
0,0,0,3,318,0,0,0,4,4,3,0,3,6,3,9,0,25,2,30
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,30
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,30


In [9]:
data=chka
data=data.query("final_score>'0'")
data.to_csv('chka_data.csv')
data_y= data['final_score']
data_x = data.drop(['final_score'], axis=1)

In [10]:
data

,Cumulative_num_days,Problem_num_day,active_num_days,active_sum_count,homework_complete_num,homework_num,num_complete,num_watched,pause_video_sum,seek_video_sum,stop_video_sum,video_backward_seek_sum,video_forward_seek_sum,video_num_days,video_pause_sum,video_stop_sum,video_sum_count,week,final_score
0,0,0,3,318,0,0,0,4,4,3,0,3,6,3,9,0,25,2,30
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,30
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,30
3,0,0,2,372,0,0,0,2,2,2,0,7,13,2,19,0,41,3,30
4,0,0,1,181,0,0,0,2,2,1,0,0,3,1,6,0,12,4,30
5,0,0,1,822,0,0,6,10,10,8,0,12,28,1,24,0,84,1,30
6,0,0,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,3,40
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,40
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,40
9,0,0,1,231,0,0,0,0,0,0,0,0,0,0,0,0,0,2,40


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.3)

In [12]:
mlp = KerasRegressor(build_fn=knn_baseline_model, epochs=1000, batch_size=2, verbose=0)

In [13]:
mlp.fit(X_train, y_train) 

In [14]:
result_dict = model_evaluate (mlp, X_train, X_test, y_train, y_test)
print(result_dict)

{'rmse': 24.807412073899446, 'mape': 47.97407319325189, 'r2': 0.23807848705363388, 'auc': 0.7658837209302326, 'training_loss': 26.973034319718735, 'f_test': 9.558542285117428, 'p': '***', 'y_pred': array([19.446613, 27.061071, 27.578533, 18.807848, 29.82264 , 46.52631 ,
       28.584412, 23.191029, 17.462534, 23.211632, 18.807848, 16.538395,
       19.468899, 18.807848, 35.158115, 18.807848, 16.538395, 23.19363 ,
       17.462534, 17.462534, 41.30568 , 22.468533, 19.998898, 31.570385,
       21.382296, 20.985203, 22.375961, 19.90235 , 17.462534, 22.13873 ,
       18.98019 , 18.380272, 45.71235 , 21.812614, 42.142353, 27.704952,
       35.021313, 16.538395, 38.415382, 16.538395, 22.429964, 17.462534,
       16.538395, 24.65242 , 18.259209, 25.696499, 37.114056, 20.759789,
       23.226143, 40.842014, 27.775372, 23.077229, 30.678455, 34.389477,
       18.807848, 16.538395, 31.35553 , 23.800385, 24.810501, 31.234924,
       39.453354, 24.831215, 16.538395, 28.797043, 26.989946, 20.999966,